This notebook requires a **MusixMatch** and **Genius** API keys. These values must be stored in a python file named *api_keys.py* and must be stored under the variables `musixmatch_api_key` and `genius_api_key`, respectively.

**However**, the Genius API requests executed in this notebook do not need a working API key, but is instead only used to initialize the API object. Therefore, the `genius_api_key` can either be set as a valid API key or as any random string.

In [1]:
from api_tools import *

# Collect data from MusixMatch

## Get the parent genres from MusixMatch

This returns a list of JSON objects, containing the genre_id and genre_name of each genre.

In [2]:
genres = get_genres()
len(genres)

48

Remove genres that are not useful for the data science project (e.g. genres of other countries' music (K-Pop, French Pop, Chinese) and 'Instrumental')

In [9]:
to_remove_genres = [3, 10, 12, 25, 27, 28, 29, 30, 51, 52, 53, 1122, 1197, 1290, 1291, 100024, 50000063, 50000064, 
                    50000061, 50000066, 50000068, 1232, 1243, 1262]

genres = [x for x in genres if x['genre_id'] not in to_remove_genres]
len(genres)

24

## Collect songs from each genre

The function `get_top_tracks()` returns a list of at most 100 JSON objects (tracks), unless otherwise specified (`page_size` parameter). 

This data project aims to collect at least 200 lyrics for each genre, thus the number of pages to be scraped is three (3), for extra allowance. The API requests begin at page three to check if it is not empty (therefore, has more than 200 songs), and decrements until page one to minimize the number of API requests.

The default values of the function parameters are based on the project's scope, which are:

> `f_track_release_group_first_release_date_min` = '20110101'  
> `f_track_release_group_first_release_date_max` = '20201231'  
> `f_lyrics_language` = 'tl'  
> `page_size` = 100  
> `s_track_rating` = 'desc'

Additionally, if the number of songs collected are less than 200, then it is not included in the DataFrame.

The genres that met the requirement are the following:

In [2]:
with open('CSV Files/genres_collection.json') as infile:
    genres_collection = json.load(infile)

# Create the DataFrames

## Import existing CSV files to be updated, if available

In [7]:
try:
    pop_df = pd.read_csv('CSV Files/pop_music.csv').drop('Unnamed: 0', axis=1)
    pop_json = json.loads(pop_df.to_json(orient='records'))

    # Convert strings to list    
    for song in pop_json:
        song['genre_id'] = song['genre_id'][1:-1].split(', ')
        song['genre_names'] = [x[1:-1] for x in song['genre_names'][1:-1].split(', ')]

except: 
    pop_json = []

In [8]:
try:
    rbSoul_df = pd.read_csv('CSV Files/rbSoul_music.csv').drop('Unnamed: 0', axis=1)
    rbSoul_json = json.loads(rbSoul_df.to_json(orient='records'))
    
    # Convert strings to list
    for song in rbSoul_json:
        song['genre_id'] = song['genre_id'][1:-1].split(', ')
        song['genre_names'] = [x[1:-1] for x in song['genre_names'][1:-1].split(', ')]
        
except:
    rbSoul_json = []

In [9]:
try:
    hiphop_df = pd.read_csv('CSV Files/hiphop_music.csv').drop('Unnamed: 0', axis=1)
    hiphop_json = json.loads(hiphop_df.to_json(orient='records'))

    # Convert strings to list
    for song in hiphop_json:
        song['genre_id'] = song['genre_id'][1:-1].split(', ')
        song['genre_names'] = [x[1:-1] for x in song['genre_names'][1:-1].split(', ')]

except:
    hiphop_json = []

In [10]:
try:
    alt_df = pd.read_csv('CSV Files/alternative_music.csv').drop('Unnamed: 0', axis=1)
    alt_json = json.loads(alt_df.to_json(orient='records'))

    # Convert strings to list
    for song in alt_json:
        song['genre_id'] = song['genre_id'][1:-1].split(', ')
        song['genre_names'] = [x[1:-1] for x in song['genre_names'][1:-1].split(', ')]
        
except:
    alt_json = []

In [11]:
try:
    rock_df = pd.read_csv('CSV Files/rock_music.csv').drop('Unnamed: 0', axis=1)
    rock_json = json.loads(rock_df.to_json(orient='records'))

    # Convert strings to list
    for song in rock_json:
        song['genre_id'] = song['genre_id'][1:-1].split(', ')
        song['genre_names'] = [x[1:-1] for x in song['genre_names'][1:-1].split(', ')]

except:
    rock_json = []

In [12]:
try:
    christian_df = pd.read_csv('CSV Files/christian_music.csv').drop('Unnamed: 0', axis=1)
    christian_json = json.loads(christian_df.to_json(orient='records'))

    for song in christian_json:
        song['genre_id'] = song['genre_id'][1:-1].split(', ')
        song['genre_names'] = [x[1:-1] for x in song['genre_names'][1:-1].split(', ')]
        
except:
    christian_json = []

## Format the collected data from MusixMatch and obtain the lyrics

In [18]:
all_music_df = pd.read_csv('CSV Files/all_music.csv').drop('Unnamed: 0', axis=1)

In [4]:
get_genre_songs(pop_json, genres_collection[0])
pop_df = pd.DataFrame(pop_json)
# pop_df.to_csv('CSV Files/pop_music.csv')

print(pop_df['lyrics'].isna().sum(), "lyrics missing out of", len(pop_df), "in total.")
pop_df

0 lyrics missing out of 300 in total.


,track_id,track_name,artist_name,genre_id,genre_names,lyrics,lyrics_source
0,163709256,Ikaw At Ako,Moira Dela Torre feat. Jason Marvin,[14],[Pop],Sabi nila\r\nBalang araw darating\r\nAng iyong...,None
1,167116583,Hanggang Dito Na Lang,Jaya,[14],[Pop],Di ko alam kung tama ba ito.\r\nLilisan ako up...,None
2,114473971,Ikaw,Yeng Constantino,[14],[Pop],Sa pagpatak ng bawat oras ay ikaw\r\nAng iniis...,None
3,114797567,Chinito,Yeng Constantino,[14],[Pop],Mapapansin mo ba\r\nKaya ang tulad ko\r\nKahit...,None
4,160018049,Dati (Cover Version),Ben&Ben,[14],[Pop],Datirati sabay pa nating pinangarap ang lahat\...,None
...,...,...,...,...,...,...,...
295,116159495,Bitter Song,Callalily feat. Maysh Baay,[14],[Pop],Di ako bitter\r\nKung akala mo'y nasasaktan\r\...,None
296,146162702,Trio,IMAGO,[14],[Pop],\r\n\r\nSaan napadala\r\nBuong maghapon naglib...,None
297,86029897,San Ka Galing Kagabi (Bonus Track),Mayonnaise,[14],[Pop],"Kung kailan tayo nagtapo, biglang binawi\nKung...",None
298,157224822,Alam Na,IMAGO,[14],[Pop],Mapaglarong lumilipad\nAng utak ko'y 'di nakik...,None


In [12]:
get_genre_songs(rbSoul_json, genres_collection[1])
rbSoul_df = pd.DataFrame(rbSoul_json)
rbSoul_df.to_csv('CSV Files/rbSoul_music.csv')

print(rbSoul_df['lyrics'].isna().sum(), "lyrics missing out of", len(rbSoul_df), "in total.")
rbSoul_df

0 lyrics missing out of 239 in total.


,track_id,track_name,artist_name,genre_id,genre_names,lyrics,lyrics_source
0,182569161,Lykkelig (Acoustic),D.Sound,"[1066, 1057, 15]","[New Acoustic, Downtempo, R&B/Soul]",Pag sapit ng dilim\nIiwanan ang mga alalahanin...,MusixMatch
1,159046130,Sa Susunod Na Lang,PDL feat. Skusta Clee & Yuri,[15],[R&B/Soul],Ha! Gusto ko lang naman makausap ka eh.\r\nYou...,None
2,43600153,Bespren,Coach Jungee feat. Yeng Constantino,[15],[R&B/Soul],Bestfriend (Bespren)\nAng hirap mong hanapin (...,MusixMatch
3,189995224,Exchange Gift,ALLMO$T,[15],[R&B/Soul],"Naaalala mo pa ba ko\r\nOh, pwede bang bati na...",None
4,157573520,Rubberband,ALLMO$T feat. FTD,"[15, 18]","[R&B/Soul, Hip Hop/Rap]",'Di ba sabi mo noon\r\nWalang bibitaw saating ...,None
...,...,...,...,...,...,...,...
234,198107668,Bakit Wala Na (feat. Chano),Peter Miranda,[15],[R&B/Soul],Bakit wala na\r\nAno na nangyari?\r\nBakit wal...,None
235,205521123,Maganda ang giseng,Eric Ruckus,[15],[R&B/Soul],Yeah\nYupyup\nRapskallion\n\nMaganda ang gisen...,MusixMatch
236,196719526,"Stardust, Vol. 2",Yalien Dahlen,[15],[R&B/Soul],"Sana'y wag na ngang lumisan pa, pagkat ito'y d...",MusixMatch
237,204549151,Pinaglayo (feat. Kxle & Kyra),Poisonhxrzy,[15],[R&B/Soul],Ako'y nalinlang ng iyong galawan\nHindi ko mal...,MusixMatch


In [13]:
get_genre_songs(hiphop_json, genres_collection[2])
hiphop_df = pd.DataFrame(hiphop_json)
hiphop_df.to_csv('CSV Files/hiphop_music.csv')

print(hiphop_df['lyrics'].isna().sum(), "lyrics missing out of", len(hiphop_df), "in total.")
hiphop_df

0 lyrics missing out of 300 in total.


,track_id,track_name,artist_name,genre_id,genre_names,lyrics,lyrics_source
0,126834098,Onoff,Eevee,"[34, 18]","[Music, Hip Hop/Rap]",O h'wag kang tumingin\nNg ganyan sa akin\n'Wag...,MusixMatch
1,125921810,lily,Eevee,[18],[Hip Hop/Rap],"Bahay kubo, kahit munti\nAng halaman doon ay s...",MusixMatch
2,169917164,Kailan Kaya Ako?,Kiara feat. Matthaios,"[34, 18]","[Music, Hip Hop/Rap]",Matthaios be wonderin'\n\nTinatanong ko sa sar...,MusixMatch
3,160904741,Ewan Ko Ba,ALLMO$T feat. Crakky,[18],[Hip Hop/Rap],Palagi ng ikaw ang laman\nNg isip ko sa tuwing...,MusixMatch
4,177235378,Kahit Na,ALLMO$T feat. Roberto Bello,[18],[Hip Hop/Rap],"Ayan na naman\r\nNakasimangot, tampo’y ‘di mai...",None
...,...,...,...,...,...,...,...
295,69144184,A.T. We On Parlay,AT feat. Antonio Maxie,[18],[Hip Hop/Rap],"Laging nananabik sa yakap mo,\nSayong pagbabal...",MusixMatch
296,155870686,Husgado,Apoc the Death Architect,[18],[Hip Hop/Rap],"Husgado - Apoc\n\nAng nasasakdal, ang pangalan...",MusixMatch
297,133336466,Love,Leon Marin,[18],[Hip Hop/Rap],"Bakit ka nababahala, gustong malaman\nHalika d...",MusixMatch
298,115635434,Warum ich Musik mach,O.R.C.A,[18],[Hip Hop/Rap],"Nakakabaliw na isipin ka, di magkasundo parang...",MusixMatch


In [19]:
get_genre_songs(alt_json, genres_collection[3])
alt_df = pd.DataFrame(alt_json)
alt_df.to_csv('CSV Files/alternative_music.csv')

print(alt_df['lyrics'].isna().sum(), "lyrics missing out of", len(alt_df), "in total.")
alt_df

0 lyrics missing out of 300 in total.


,track_id,track_name,artist_name,genre_id,genre_names,lyrics,lyrics_source
0,206436336,Paubaya,Moira Dela Torre,[20],[Alternative],Saan nagsimulang magbago ang lahat\r\nKailan n...,None
1,152025019,Buwan,Juan Karlos Labajo,[20],[Alternative],"Ako'y sayo, ikaw ay akin\r\nGanda mo sa paning...",None
2,88489270,Tadhana,Up Dharma Down,[20],[Alternative],Sa hindi inaaasahang\r\nPagtatagpo ng mga mund...,None
3,78880323,Orange,Parokya Ni Edgar,"[1133, 20]","[Pop/Rock, Alternative]","Gusto kong kumain ng lemon,\r\nKahit ano, kahi...",None
4,170029087,Pagtingin,Ben&Ben,"[20, 14]","[Alternative, Pop]",Dami pang gustong sabihin\r\nNgunit wag nalang...,None
...,...,...,...,...,...,...,...
295,129569951,Lutang,Ely Buendia feat. the itchyworms,[20],[Alternative],"Nakatulala, nawawala\n'Di makausap, parang nam...",MusixMatch
296,188322731,Ipanumpa ko,Oh! Caraga,[20],[Alternative],"Ipanumpa kong di t'ka biyaan,\nUg pagahuktan k...",MusixMatch
297,112026444,Silakbo,MilesExperience,[20],[Alternative],Ang hindi sadyang samahan\nBigla na lang nagtu...,MusixMatch
298,178700922,Sigurado,Udd,"[7, 20]","[Electronic, Alternative]",Natatakot ikaw ay mawala sa aking tabi\r\nNamu...,None


In [9]:
get_genre_songs(rock_json, genres_collection[4])
rock_df = pd.DataFrame(rock_json)
rock_df.to_csv('CSV Files/rock_music.csv')

print(rock_df['lyrics'].isna().sum(), "lyrics missing out of", len(rock_df), "in total.")
rock_df

0 lyrics missing out of 300 in total.


,track_id,track_name,artist_name,genre_id,genre_names,lyrics,lyrics_source
0,136802912,Kahit Di Mo Alam,December Avenue,[21],[Rock],Ipikit mo man ang iyong mata\r\n'Di pa rin nam...,None
1,84257639,Parang Mali,Siakol,[21],[Rock],\r\n\r\nParang Mali\r\nKung kinausap mo lang s...,None
2,104448893,Eroplanong Papel,December Avenue,[21],[Rock],"Sandali, 'wag kang mapupuno sa paghihirap\r\nD...",None
3,84257635,Basag,Siakol,[21],[Rock],\r\n\r\nKung hindi na masaya\r\nKung hindi na ...,None
4,161144075,Bangin,Mayonnaise,[21],[Rock],Bumabalot sa katawan mo ng halos isang taon\nA...,None
...,...,...,...,...,...,...,...
295,149147068,Huli,Viente,[21],[Rock],Biglang tumunog\nAng aking telepono\nDaliang s...,None
296,110219982,Sayo'ng Sa'yo,Zoom Zoom Lunacy,[21],[Rock],Yan ang sabi mo sa 'kin\nYan ang sabi mo sa 'k...,None
297,162636949,Antok,Agaw Agimat,[21],[Rock],"Pauwi ka na, ako'y papasok pa lang\nWala na na...",None
298,124514919,Lo-Fi,Pinoy Rock Station,[21],[Rock],Kay bilis ng pagbabago\nHindi ko na masabayan\...,None


In [171]:
get_genre_songs(christian_json, genres_collection[5])
christian_df = pd.DataFrame(christian_json)
christian_df.to_csv('CSV Files/christian_music_noAZLyrics2.csv')

print(christian_df['lyrics'].isna().sum(), "lyrics missing out of", len(christian_df), "in total.")
christian_df

12 lyrics missing out of 293 in total.


,track_id,track_name,artist_name,genre_id,genre_names,lyrics,lyrics_source
0,201311852,Walang Katulad,Victory Worship,[22],[Christian & Gospel],Ika'y aking liwanag\r\nSa dilim ng landas\r\nS...,None
1,73676809,Shine Upon the Philippines,Victory Worship,[22],[Christian & Gospel],"You are the light, You are the hope\r\nYou are...",None
2,189377564,Pagbabalik,Victory Worship,[22],[Christian & Gospel],"Ngayon ay aahon, at kakalimutan ang nakaraan\r...",None
3,189377568,Tagumpay,Victory Worship,[22],[Christian & Gospel],"Sa sigaw ng alon, 'di mangangamba\r\nSa gitna ...",None
4,189377567,Maghari,Victory Worship,[22],[Christian & Gospel],"Sa gitna ng kaguluhan, ang tinig Mo ay hanap\r...",None
...,...,...,...,...,...,...,...
288,83929781,Gagawa Ang Diyos (God Will Make a Way),Lito Magnaye,[22],[Christian & Gospel],Gagawa ang Diyos\nNg daan sa buhay ko\nPagkilo...,None
289,79505181,Ang Panginoon Ay Awitan,Tony Rodeo,[22],[Christian & Gospel],Ang panginoon ay awitan\nSa kanya ang kadakila...,None
290,136863016,Ikaw Lamang (Live),Rommel Guevara,[22],[Christian & Gospel],Oh Diyos sa kabutihan mo\nAkoy naririto\nUpang...,None
291,78391977,Mga Pangako Mo (Live),Jesus One Generation,[22],[Christian & Gospel],Sa'Yo umaasa\nKaibigang tapat at sandigan\nSa'...,None


### Manual Cleaning

In [29]:
# Check for songs with translated sections in the lyrics

withTrans = []
music_dfs = [pop_df, rbSoul_df, hiphop_df, alt_df, rock_df, christian_df]

for df in music_dfs:
    withTrans.append(df[df['lyrics'].notnull()]
                     [df[df['lyrics'].notnull()]['lyrics'].str.lower().str.contains("translation")]
                     .index.tolist())
    
withTrans

[[43, 70], [], [], [], [], []]

In [20]:
# Remove translated sections manually

lyrics = pop_df.iloc[70]['lyrics']
pop_df.loc[70, 'lyrics'] = lyrics[lyrics.index('Tagalog')+7:lyrics.index('English Translation', 20)]

lyrics = pop_df.iloc[43]['lyrics']
pop_df.loc[43, 'lyrics'] = lyrics[:lyrics.index('English Translation')]

pop_df.to_csv('CSV Files/pop_music.csv')

In [ ]:
# Check for songs that have no genre_ids or has string IDs

for i in range(len(all_music_json)):
    song = all_music_json[i]
    
    if (len(song['genre_id']) == 0 or type(song['genre_id'][0]) != int):
        print(song['track_id'], song['track_name'], song['genre_id'], song['genre_names'], i)

In [154]:
# Fixing genre_ids that have extraneous characters (e.g. '''//''22''//''') 
# and converting all string IDs to integer

for x in christian_json:
    if type(x['genre_id'][0]) != int:
        x['genre_id'] = [int(re.sub('[^0-9]*', '', i)) for i in x['genre_id']]

## Compile all genres into one DataFrame

In [91]:
all_music_json = pop_json.copy()
all_music_json.extend(rbSoul_json)
all_music_json.extend(hiphop_json)
all_music_json.extend(alt_json)
all_music_json.extend(rock_json)
all_music_json.extend(christian_json)

In [92]:
all_music_df = pd.DataFrame(all_music_json)
all_music_df.to_csv('CSV Files/all_music.csv')
all_music_df

,track_id,track_name,artist_name,genre_id,genre_names,lyrics,lyrics_source
0,163709256,Ikaw At Ako,Moira Dela Torre feat. Jason Marvin,[14],[Pop],Sabi nila\r\nBalang araw darating\r\nAng iyong...,None
1,167116583,Hanggang Dito Na Lang,Jaya,[14],[Pop],Di ko alam kung tama ba ito.\r\nLilisan ako up...,None
2,114473971,Ikaw,Yeng Constantino,[14],[Pop],Sa pagpatak ng bawat oras ay ikaw\r\nAng iniis...,None
3,114797567,Chinito,Yeng Constantino,[14],[Pop],Mapapansin mo ba\r\nKaya ang tulad ko\r\nKahit...,None
4,160018049,Dati (Cover Version),Ben&Ben,[14],[Pop],Datirati sabay pa nating pinangarap ang lahat\...,None
...,...,...,...,...,...,...,...
1727,83929781,Gagawa Ang Diyos (God Will Make a Way),Lito Magnaye,[22],[Christian & Gospel],Gagawa ang Diyos\nNg daan sa buhay ko\nPagkilo...,MusixMatch
1728,79505181,Ang Panginoon Ay Awitan,Tony Rodeo,[22],[Christian & Gospel],Ang panginoon ay awitan\nSa kanya ang kadakila...,MusixMatch
1729,136863016,Ikaw Lamang (Live),Rommel Guevara,[22],[Christian & Gospel],Oh Diyos sa kabutihan mo\nAkoy naririto\nUpang...,MusixMatch
1730,78391977,Mga Pangako Mo (Live),Jesus One Generation,[22],[Christian & Gospel],Sa'Yo umaasa\nKaibigang tapat at sandigan\nSa'...,MusixMatch


In [168]:
pop_df = pd.DataFrame(pop_json)
rbSoul_df = pd.DataFrame(rbSoul_json)
hiphop_df = pd.DataFrame(hiphop_json)
alt_df = pd.DataFrame(alt_json)
rock_df = pd.DataFrame(rock_json)
christian_df = pd.DataFrame(christian_json)

pop_df.to_csv('CSV Files/pop_music.csv')
rbSoul_df.to_csv('CSV Files/rbSoul_music.csv')
hiphop_df.to_csv('CSV Files/hiphop_music.csv')
alt_df.to_csv('CSV Files/alternative_music.csv')
rock_df.to_csv('CSV Files/rock_music.csv')
christian_df.to_csv('CSV Files/christian_music.csv')
all_music_df.to_csv('CSV Files/all_music.csv')

In [18]:
print("Number of Songs with Lyrics")
print()

print("Pop:                 ", pop_df['lyrics'].notnull().sum(),       "  out of   ", len(pop_df))
print("R&B / Soul:          ", rbSoul_df['lyrics'].notnull().sum(),    "  out of   ", len(rbSoul_df))
print("Hiphop:              ", hiphop_df['lyrics'].notnull().sum(),    "  out of   ", len(hiphop_df))
print("Alternative:         ", alt_df['lyrics'].notnull().sum(),       "  out of   ", len(alt_df))
print("Rock:                ", rock_df['lyrics'].notnull().sum(),      "  out of   ", len(rock_df))
print("Christian & Gospel:  ", christian_df['lyrics'].notnull().sum(), "  out of   ", len(christian_df))
print("Total:              ",  all_music_df['lyrics'].notnull().sum(), "  out of  ",  len(all_music_df))

Number of Songs with Lyrics

Pop:                  300   out of    300
R&B / Soul:           239   out of    239
Hiphop:               300   out of    300
Alternative:          300   out of    300
Rock:                 300   out of    300
Christian & Gospel:   281   out of    293
Total:               1720   out of   1732


In [3]:
test_json = []

In [6]:
get_genre_songs(test_json, genres_collection[0], 290)

Pop
	song  300 / 300 with 9 second delay
	songs without lyrics: 0

	collecting lyrics from MetroLyrics: found


In [8]:
test_df = pd.DataFrame(test_json)
test_df

,track_id,track_name,artist_name,genre_id,genre_names,lyrics,lyrics_source
0,43136043,Sino Nga Ba Siya,Sarah Geronimo,[14],[Pop],'Di ko inisip na mawawala ka pa\nAkala ko'y pa...,Genius
1,72465125,Susan,Mayonnaise,"[20, 14]","[Alternative, Pop]",\n\r\nSusan\nNa'san ka\nHindi mo ba alam\nAng ...,AZLyrics
2,43329172,"Hindi Na Magbabago (From ""Kahit Puso'y Masugat...",Erik Santos,[14],[Pop],Hindi mapaliwanag ng makilala ka\nMay binulong...,Genius
3,116062255,Ilang Pasko Pa Ba,Juris,"[1080, 14]","[Christmas, Pop]",,MetroLyrics
4,114724404,Patuloy Ang Pangarap - Alternate Version of Mi...,Angeline Quinto,[14],[Pop],'Di pa rin makapaniwala Sa lahat ng nangyari\n...,Genius
5,114769649,Takbo Ng Mundo,Erik Santos,[14],[Pop],\n\r\nMinsan kung alin ang di mo ninanais\nAng...,AZLyrics
6,114769653,"Sana Naman, Sana Na Nga",Erik Santos,[14],[Pop],I.\nAlam kung minamahal mo ako ng buong tapat\...,MusixMatch
7,130181219,Halik Na Lang,KZ Tandingan,[14],[Pop],Akala ko din noong una\nMalabo mangyari\nPero ...,Genius
8,114193484,Tanging Ligaya Ko,Michael Pangilinan,[14],[Pop],"Nung una tayong nagkita, ako'y nabighani na\nK...",Genius
9,116158452,"Wag Ka Nang Umiyak (From ""The Trial"")",KZ Tandingan,[14],[Pop],"Wag ka nang umiyak, sa mundong pabago-bago\npa...",MetroLyrics
